In [ ]:
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, DAILY
from tqdm.notebook import tqdm

In [ ]:
from cursdb import CursDB
db = CursDB("bnr.xml.db")


In [ ]:
from cursclient import CursClient, date2dt
client = CursClient()


In [ ]:
lastdate = client.lastdate


In [ ]:
client.getall("2023-01-29")

In [ ]:
client._client.service.getvalue(date2dt("1999-01-29"), "USD")

In [ ]:
lastdate = client.lastdate
for days in range(30):
    try:
        when = lastdate - relativedelta(days=days)
        rates = client.getall(when)
    except:
        print(when)
        raise

    for date, currency, value in rates:
        db.insert_value(date, currency, value, replace=False)
    db.commit()



In [ ]:
list(db.select_rows(date=('2022-01-01', "2022-01-31"), currency="XDR"))

In [ ]:
currencies = list(map(lambda x: x[1], client.getall()))
print(*currencies, sep=", ")

In [ ]:
try:
    days = list(rrule(DAILY, date2dt("2010-01-01"), until=client.lastdate))
    days.reverse()
    loop = tqdm(days)
    inserted = 0
    for date in loop:
        for currency in currencies:
            if (value := db.get_value(date, currency)) is None:
                loop.set_postfix_str(f"{date.date()} {currency}")
                value = client.getvalue(date, currency)
                if value is not None:
                    db.insert_value(date, currency, value, replace=False)
                    inserted += 1
                    if inserted > 100:
                        db.commit()
                        inserted = 0
except KeyboardInterrupt:
    pass
finally:
    db.commit()
